<a href="https://colab.research.google.com/github/hivancd/ehealth-entity-recognition-problem/blob/franco/myTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Innit

In [ ]:
#Innit
! pip install gliner
! pip install accelerate -U
! git clone https://github.com/hivancd/ehealth-entity-recognition-problem
! git clone https://github.com/ehealthkd/corpora
import sys
sys.path.append('ehealth-entity-recognition-problem')
from anntools import Collection
from pathlib import Path
! pip install spacy
! python -m spacy download es_core_news_sm
import spacy

### Loading trainning data

In [ ]:
c = Collection()
c.load(Path('/content/ehealth-entity-recognition-problem/data/training/scenario.txt'))

In [ ]:
# def get_ner(tokens:list,keyphrases):



In [ ]:
# HIVAN
tokenizer = spacy.load("es_core_news_sm")
data=[]
in_dict={'tokenized_sentence':[],'ner':[]}
# for sentence in c.sentences:
#   text= sentence.text
#   tokens = tokenizer(text)
#   in_dict['tokenized_sentence']= tokens

s=c.sentences[0]
t=tokenizer()
kp=s.keyphrases[0]
print(kp.spans)
# for t in tokens:

[(51, 61)]


### Testing/Evaluating data

To compute the scores, we define correct, partial, missing, incorrect, and spurious matches. The expected and actual output files do not need to agree on the ID for each entity, nor on their order. The evaluator matches are based on the start and end of text spans and the corresponding type. A brief description about the metrics follows:

- **Correct matches** are reported when a text in the dev file matches exactly with a corresponding text span in the gold file in START and END values, and also the entity type. Only one correct match per entry in the gold file can be matched. Hence, duplicated entries will count as Spurious.
- **Incorrect matches** are reported when START and END values match, but not the type.
- **Partial matches** are reported when two intervals [START, END] have a non-empty intersection, such as the case of “vías respiratorias” and “respiratorias” in the previous example (and matching LABEL). Notice that a partial phrase will only be matched against a single correct phrase. For example, “tipo de cáncer” could be a partial match for both “tipo” and “cáncer”, but it is only counted once as a partial match with the word “tipo”. The word “cancer” is counted then as Missing. This aims to discourage a few large text spans that cover most of the document from getting a very high score.
- **Missing matches** are those that appear in the gold file but not in the dev file.
- **Spurious matches** are those that appear in the dev file but not in the gold file.

From these definitions, we compute precision, recall, and a standard F1 measure as follows:

$$
\text{Rec}_{A} = \frac{C_A + \frac{1}{2} P_A}{C_A + I_A + P_A + M_A}
$$

$$
\text{Prec}_{A} = \frac{C_A + \frac{1}{2} P_A}{C_A + I_A + P_A + S_A}
$$

$$
F_{1A} = 2 \cdot \frac{\text{Prec}_{A} \cdot \text{Rec}_{A}}{\text{Prec}_{A} + \text{Rec}_{A}}
$$

A higher precision means that the number of spurious identifications is smaller compared to the number of missing identifications, and a higher recall means the opposite. Partial matches are given half the score of correct matches, while missing and spurious identifications are given no score.

F1 will determine the ranking.

example:

correct_A: 813

incorrect_A: 75

partial_A: 65

spurious_A: 674

missing_A: 352

recall: 0.3776

precision: 0.4773

f1: 0.4217

In [28]:
import re

def read_file_as_string(file_path):
    # Ensure the file has the correct extension
    if not file_path.endswith(('.txt', '.ann')):
        raise ValueError("File must be a .txt or .ann file")

    # Open the file and read its content
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            return content
    except Exception as e:
        # Handle possible exceptions (e.g., file not found)
        raise IOError(f"Error reading file {file_path}: {str(e)}")

def evaluate_annotations(gold_annotations, dev_annotations):
    
    # Convert annotations to a more manageable format
    gold_dict = {f"{ann.split()[2]} {ann.split()[3]}": ann.split()[1] for ann in gold_annotations.split('\n') if ann}
    dev_dict = {f"{ann.split()[2]} {ann.split()[3]}": ann.split()[1] for ann in dev_annotations.split('\n') if ann}
    
    # Initialize counters
    correct_matches = 0
    incorrect_matches = 0
    partial_matches = 0
    missing_matches = len(gold_dict)
    spurious_matches = 0
    
    matched_spans = set()  # Keep track of spans that have been matched as partial

    for span, type_ in gold_dict.items():
        if span in dev_dict:
            if dev_dict[span] == type_:
                correct_matches += 1
            else:
                incorrect_matches += 1
            missing_matches -= 1
        else:
            #Check partial matches
            gold_start, gold_end = map(int, span.split())
            for dev_span, dev_type in dev_dict.items():
                if dev_span in matched_spans:  # Skip if this span is already matched as part of a larger span
                    continue
                dev_start, dev_end = map(int, dev_span.split())
                if not (gold_end < dev_start or gold_start > dev_end) and dev_type == type_:
                    partial_matches += 1
                    missing_matches -= 1
                    matched_spans.add(dev_span)  # Mark this span as matched to avoid double-counting
                    break

    # Count spurious matches
    spurious_matches = len(dev_annotations.split('\n')) - correct_matches - incorrect_matches - partial_matches

    # Calculate metrics
    recall = (correct_matches + 0.5 * partial_matches) / (correct_matches + incorrect_matches + partial_matches + missing_matches)
    precision = (correct_matches + 0.5 * partial_matches) / (correct_matches + incorrect_matches + partial_matches + spurious_matches)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return f"""
"correct_A": {correct_matches}
"incorrect_A": {incorrect_matches}
"partial_A": {partial_matches}
"spurious_A": {spurious_matches}
"missing_A": {missing_matches}
--------------------
"recall": {recall}
"precision": {precision}

"f1": {f1}"""

def score(gold_file_path, dev_file_path):
    # Read the gold and dev annotations as strings
    gold_annotations = read_file_as_string(gold_file_path)
    dev_annotations = read_file_as_string(dev_file_path)

    # Evaluate the annotations and return the results
    print(evaluate_annotations(gold_annotations, dev_annotations))

#example of use with the scenario_example.ann and predictions_example.ann files in the root of the repository
score("C:\Franco\Proyects\JackRipper01\ehealth-entity-recognition-problem\scenario_example.ann","C:\Franco\Proyects\JackRipper01\ehealth-entity-recognition-problem\predictions_example.ann")



"correct_A": 14
"incorrect_A": 0
"partial_A": 0
"spurious_A": 103
"missing_A": 6
--------------------
"recall": 0.7
"precision": 0.11965811965811966

"f1": 0.20437956204379562


## Mauricio

In [ ]:
a = [[print("'"+x["text"]+"'"), [print(b.text,"->", b.label) for b in x.keyphrases],print()] for x in c.sentences[:10]]

TypeError: 'Sentence' object is not subscriptable